In [0]:
#Employees earning more than department average salary

In [0]:
from pyspark.sql.functions import col,avg

#sample data
data=[
    (1,"Alice","HR",60000),
    (2,"Bob","HR",50000),
    (3,"Charlie","Finance",70000),
    (4,"David","Finance",75000),
    (5,"Eve","Engineering",90000),
    (6,"Frank","Engineering",93000),
    (7,"Grace","HR",45000),
    (8,"Hank","Engineering",98000),
    (9,"Ivy","Finance",66000),
]

columns=["employee_id","employee_name","department","salary"]

#create dataframe:
df = spark.createDataFrame(data,columns)

df.display()

employee_id,employee_name,department,salary
1,Alice,HR,60000
2,Bob,HR,50000
3,Charlie,Finance,70000
4,David,Finance,75000
5,Eve,Engineering,90000
6,Frank,Engineering,93000
7,Grace,HR,45000
8,Hank,Engineering,98000
9,Ivy,Finance,66000


In [0]:
from pyspark.sql.window import Window

windowSpec = Window.partitionBy("department")

df = df.withColumn("avg_dept_salary",avg(col("salary")).over(windowSpec))

result_df = df.filter(col("salary") > col("avg_dept_salary")) #.select("employee_id")

result_df.show()

+-----------+-------------+-----------+------+------------------+
|employee_id|employee_name| department|salary|   avg_dept_salary|
+-----------+-------------+-----------+------+------------------+
|          8|         Hank|Engineering| 98000| 93666.66666666667|
|          4|        David|    Finance| 75000| 70333.33333333333|
|          1|        Alice|         HR| 60000|51666.666666666664|
+-----------+-------------+-----------+------+------------------+



%md 
###using Spark SQL

In [0]:
df.createOrReplaceTempView("employees")

SQL Window Function:

In [0]:
%sql

with cte as (
select e.employee_id,e.employee_name,e.department,e.salary,
avg(salary) over (partition by department) as avg_dept_salary
from employees e
)

select *
from cte
where salary > avg_dept_salary

employee_id,employee_name,department,salary,avg_dept_salary
8,Hank,Engineering,98000,93666.66666666667
4,David,Finance,75000,70333.33333333333
1,Alice,HR,60000,51666.666666666664


Correlated Subquery:

In [0]:
%sql
select *
from employees e1
where e1.salary > (select avg(salary) from employees e2 where e2.department = e1.department)

--condition WHERE e2.department = e1.department, filters the records in the subquery to only include employees from the same department as the current employee (e1).


employee_id,employee_name,department,salary,avg_dept_salary
1,Alice,HR,60000,51666.666666666664
4,David,Finance,75000,70333.33333333333
8,Hank,Engineering,98000,93666.66666666667


Inner Join

In [0]:
%sql
select e1.*
from employees e1
join 
(select department,avg(salary) as avg_dept_salary from employees group by department) e2
on e1.department=e2.department
where e1.salary > e2.avg_dept_salary

employee_id,employee_name,department,salary,avg_dept_salary
1,Alice,HR,60000,51666.666666666664
4,David,Finance,75000,70333.33333333333
8,Hank,Engineering,98000,93666.66666666667
